# Join Statements in SQL

![JOIN Statements](join_statements.png "Join Statements")

In [3]:
import sqlite3 as sq3
import pandas as pd

## Create the initial table

In [4]:
create_author_table_query = """
CREATE TABLE authors(
    author_id VARCHAR(36) NOT NULL PRIMARY KEY,
    author_name VARCHAR(12) NOT NULL
)
"""

create_reviews_table_query = """
CREATE TABLE ratings(
    review_id VARCHAR(36) NOT NULL PRIMARY KEY,
    post_id VARCHAR(36) NOT NULL,
    rating INT NOT NULL
)
"""

create_table_query = """
CREATE TABLE blog_post(
    post_id VARCHAR(36) NOT NULL PRIMARY KEY,
    name VARCHAR(22) NOT NULL,
    author_id VARHCHAR(36) NOT NULL,
    FOREIGN KEY (author_id) REFERENCES authors(author_id)
)
"""

### Connect or create then connect the Database

In [7]:
def connect_to_or_create_table(table_name: str):
    try:
        print("Trying to connect to existing  table.")
        conn = sq3.connect(table_name)
    except:
        print("Creating new table")
        conn = sq3.Connection(table_name)
        conn.execute(create_author_table_query)
        conn.execute(create_reviews_table_query)
        conn.execute(create_table_query)
        conn.close()

def populate_database(table_name: str,
                      authors: list[tuple[int, str]],
                      blog_posts: list[tuple[int, str, int]]):
    """_summary_

    Args:
        authors (list): _description_
        blog_posts (list): _description_
    """
    assert len(authors) == len(blog_posts)
    conn = sq3.connect(table_name)
    cursor = conn.cursor()
    try:
        cursor = conn.cursor()
        cursor.execute("BEGIN TRANSACTION")
        cursor.executemany("INSERT INTO authors(author_id, author_name) VALUES(?, ?)", authors)
        cursor.executemany("INSERT INTO blog_post(post_id, name, author_id) VALUES(?, ?, ?)", blog_posts)
        conn.commit()
    except sq3.Error as e:
        print(f"Error populating database: {e}")
        conn.rollback()
    finally:
        cursor.close()
        conn.close()  
    

Trying to connect to existing  table.


### Populate the Table with Data

In [ ]:
TABLE_NAME = 'blog_posts'

In [8]:
authors = [('auth1', 'mark'), ('auth2', 'john')]
blog_posts = [('post1', 'Blog Post 1', authors[0][0]), ('post2', 'Blog Post 2', authors[1][0])]

In [14]:
connect_to_or_create_table(TABLE_NAME)
populate_database(table_name=TABLE_NAME, authors=authors, blog_posts=blog_posts)

## Selecting Blog Posts and Authors (Inner Join)

In [15]:
select_blog_posts_query = """
SELECT blog_post.post_id, blog_post.name, authors.* FROM blog_post
INNER JOIN authors ON blog_post.author_id = authors.author_id 
"""

In [17]:
table_name = 'blog_posts'
conn = sq3.connect(table_name)

In [18]:
pandas_generator_object = pd.read_sql(select_blog_posts_query, conn)
pandas_generator_object.head()

,post_id,name,author_id,author_name
0,post1,Blog Post 1,auth1,mark
1,post2,Blog Post 2,auth2,john


## Selecting Reviews (Left Join)

In [29]:
select_blog_posts_with_reviews = """
SELECT blog_post.post_id, blog_post.name, authors.*, ratings.rating FROM blog_post
INNER JOIN authors ON blog_post.author_id = authors.author_id
LEFT JOIN ratings ON blog_post.author_id = authors.author_id
"""

In [30]:
table_name = 'blog_posts'
conn = sq3.connect(table_name)

In [31]:
pandas_generator_object = pd.read_sql(select_blog_posts_with_reviews, conn)
pandas_generator_object.head()

,post_id,name,author_id,author_name,rating
0,post1,Blog Post 1,auth1,mark,None
1,post2,Blog Post 2,auth2,john,None
